In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=275cb0e4f49cc89a4676397c2f43d734d430a7f3c6d7ae43ec3e16a14b4ee8e4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark import SparkConf, SparkContext
import collections

sc=SparkContext()
rdd = sc.parallelize([3,4,56,7,4,2])

sq = rdd.map(lambda x: x*x)
print(sq.collect())
sc.stop()

[9, 16, 3136, 49, 16, 4]


Ratings Histogram

In [3]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext(conf = conf)

lines = sc.textFile("u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print(key, value)
sc.stop()




1 6111
2 11370
3 27145
4 34174
5 21203


In [12]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

lines = sc.textFile("1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))
sc.stop()


ITE00100554	5.36F
EZE00100082	7.70F


In [13]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("MaxTemperatures")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

lines = sc.textFile("1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMAX" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: max(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))
sc.stop()

ITE00100554	90.14F
EZE00100082	90.14F


In [14]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)

input = sc.textFile("book.txt")
words = input.flatMap(lambda x: x.split())
wordCounts = words.countByValue()

for word, count in wordCounts.items():
    print(word, count)


sc.stop()

Görüntülenen çıkış son 5000 satıra kısaltıldı.
individual. 2
top-down 1
restrictions 2
make, 1
clients. 2
computer 6
programmer, 1
conform 1
style 4
guide 1
existing 17
architecture 1
artist, 1
show 16
Working 4
what's 6
important. 8
Did 2
dress 1
code 3
yourself? 1
meeting 2
clients, 2
want. 9
maintain 9
wardrobe. 1
Personally, 2
regardless; 1
helps 4
frame 1
mind. 5
IS 1
SELF-EMPLOYMENT 1
FOR 3
YOU? 1
Self-employment 4
forms 4
cannot 9
environment. 1
unbounded 1
rewards, 2
far, 5
serious. 1
Let's 8
you? 4
answer. 1
honestly 2
assess 1
diving 2
undertaking, 1
likelihood 2
success. 8
then, 1
guarantees 1
exercises 1
chapter 1
stand. 1
unimaginably 1
well, 11
badly. 1
We 6
favor 4
not, 13
conventional 2
workforce 1
savings 7
now. 6
FLOWCHART: 1
SHOULD 1
EVEN 1
CONSIDER 1
SELF-EMPLOYMENT? 1
Minimizing 1
ensuring 3
cushion 1
expenses. 6
truly 3
betting 1
farm 5
identify 8
store 1
changes. 2
flowchart 2
crazy 1
this. 2
reading 6
device 1
display 17
main 6
points: 1
HAVING 1
FINANCIAL 2
SAF

In [17]:
import re
from pyspark import SparkConf, SparkContext

def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())

conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)

input = sc.textFile("book.txt")
words = input.flatMap(normalizeWords)

wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey(False)
results = wordCountsSorted.collect()

for result in results:
    count = str(result[0])
    word = result[1].encode('ascii', 'ignore')
    if (word):
        print(word.decode() + ":\t\t" + count)

sc.stop()



you:		1878
to:		1828
your:		1420
the:		1292
a:		1191
of:		970
and:		934
that:		747
it:		649
in:		616
is:		560
for:		537
on:		428
are:		424
if:		411
s:		391
i:		387
business:		383
can:		376
be:		369
as:		343
have:		321
with:		315
t:		301
this:		280
or:		278
time:		255
but:		242
they:		234
will:		231
what:		229
at:		220
my:		215
re:		214
do:		207
not:		203
about:		202
more:		200
product:		182
an:		178
up:		177
need:		174
them:		166
from:		166
how:		163
there:		162
out:		161
new:		153
people:		145
work:		144
so:		143
just:		142
own:		140
all:		137
don:		133
get:		123
customers:		123
by:		122
want:		122
company:		122
their:		122
some:		121
ll:		114
self:		111
website:		109
make:		108
may:		107
even:		104
when:		102
one:		100
ve:		95
than:		92
also:		91
job:		90
much:		90
who:		88
money:		86
was:		85
these:		82
find:		81
sales:		80
only:		79
into:		79
yourself:		78
other:		78
like:		78
no:		76
probably:		76
employment:		75
ads:		75
day:		73
good:		72
many:		71
before:		70
most:		70
might:		

In [18]:
from pyspark import SparkConf, SparkContext

def countCoOccurences(line):
    elements = line.split()
    return (int(elements[0]), len(elements) - 1)

def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode("utf8"))

conf = SparkConf().setMaster("local").setAppName("PolpularHero")
sc = SparkContext(conf = conf)

names = sc.textFile("Marvel-names.txt")
namesRdd = names.map(parseNames)

lines = sc.textFile("Marvel-graph.txt")

pairings = lines.map(countCoOccurences)
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)
flipped = totalFriendsByCharacter.map(lambda xy : (xy[1], xy[0]))

mostPopular = flipped.max()

mostPopularName = namesRdd.lookup(mostPopular[1])[0]

print(str(mostPopularName) + " is the most popular superhero, with " + str(mostPopular[0]) + \
      " co-appearances.")

sc.stop()

b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.


In [19]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Step 1: Initialize Spark
spark = SparkSession.builder.appName("PimaIndianClassification").getOrCreate()

# Step 2: Load the dataset
data = spark.read.csv("pima-indians-diabetes.csv", inferSchema=True, header=True)

# Step 3: Prepare the data for training
feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data).select("features", "Outcome")

# Step 4: Split the data into training and testing sets
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

# Step 5: Train a logistic regression model
lr = LogisticRegression(labelCol="Outcome", featuresCol="features")
model = lr.fit(train_data)

# Step 6: Make predictions on the test data
predictions = model.transform(test_data)

# Step 7: Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="Outcome")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8546265328874024
